In [19]:
import numpy as np
import os
import matplotlib.pyplot as plt
import enviroment_no_visual as enviroment_no_visual
import enviroment_visual as enviroment_visual
import tensorflow as tf
import keras
import random

Funzione per creare una neural network lineare 

In [20]:
class AdvantageLayer(keras.layers.Layer):
    def call(self, raw_advantages):
        return raw_advantages - tf.reduce_max(raw_advantages, axis=1, keepdims=True)

def Dueling_DQN(units):
    input_states = keras.layers.Input(shape=[11])
    hidden = keras.layers.Dense(units, activation="relu", name='HiddenLayer')(input_states)
    state_values = keras.layers.Dense(1, name='StateValueLayer')(hidden)
    raw_advantages = keras.layers.Dense(3, name='RawAdvantagesLayer')(hidden)
    advantages = AdvantageLayer(name='AdvantageCalculationLayer')(raw_advantages)
    Q_values = keras.layers.Add(name='QValueLayer')([state_values, advantages])
    model = keras.Model(inputs=[input_states], outputs=[Q_values], name='LinearQNetwork')
    return model
model = Dueling_DQN()
model.summary()

TypeError: Dueling_DQN() missing 1 required positional argument: 'units'

: 

Implementazione di un buffer circolare che permetta inserimento/cancellazione degli elementi e accesso random veloce. 

In [12]:
class ReplayBuffer:
    def __init__(self, max_size):
        self.buffer = np.empty(max_size, dtype=object)
        self.max_size = max_size
        self.index = 0
        self.size = 0

    def append(self, obj):
        self.buffer[self.index] = obj
        self.size = min(self.size + 1, self.max_size)
        self.index = (self.index + 1) % self.max_size

    def sample(self, batch_size):
        indices = np.random.randint(self.size, size=batch_size)
        return self.buffer[indices]
    
    def sample_experiences(self, batch_size):
        batch = self.sample(batch_size)
        states, actions, rewards, next_states, game_over = [
            np.array([experience[field_index] for experience in batch])
                        for field_index in range(5)]
        return states, actions, rewards, next_states, game_over

In [13]:
class PrioritizedReplayBuffer:
    def __init__(self, max_size, zeta=0.6):
        """
        Prioritized Experience Replay buffer.
        capacity: massimo numero di transizioni da memorizzare.
        alpha: grado di priorità, 0 significa nessuna priorità (PER diventa normale).
        """
        self.max_size = max_size
        self.zeta = zeta
        self.buffer = []
        self.priorities = np.zeros((max_size,), dtype=np.float32)
        self.pos = 0
    
    def add(self, transition, td_error):
        """
        Aggiunge una transizione alla memoria con priorità.
        transition: transizione (stato, azione, ricompensa, prossimo stato, done)
        td_error: l'errore TD per la transizione, utilizzato per calcolare la priorità.
        """
        max_priority = np.max(self.priorities) if self.buffer else 1.0
        if len(self.buffer) < self.max_size:
            self.buffer.append(transition)
        else:
            self.buffer[self.pos] = transition

        # Assegna la priorità basata sull'errore TD
        self.priorities[self.pos] = max_priority if td_error is None else (abs(td_error) + 1e-5) ** self.zeta
        self.pos = (self.pos + 1) % self.max_size
    
    def sample(self, batch_size, beta=0.4):
        """
        Campiona un batch di transizioni dalla memoria.
        beta: parametro per correggere il bias nel campionamento.
        """
        if len(self.buffer) == self.max_size:
            priorities = self.priorities
        else:
            priorities = self.priorities[:self.pos]
        
        # Probabilità di campionamento per ogni transizione
        probs = priorities ** self.zeta
        probs /= probs.sum()

        # Campiona gli indici con probabilità proporzionale alla priorità
        indices = np.random.choice(len(self.buffer), batch_size, p=probs)
        samples = [self.buffer[idx] for idx in indices]

        # Calcola i pesi di correzione per ciascun campione
        total = len(self.buffer)
        weights = (total * probs[indices]) ** (-beta)
        weights /= weights.max()

        return samples, indices, weights
    
    def update_priorities(self, batch_indices, batch_td_errors):
        """
        Aggiorna le priorità nel buffer basate sui nuovi TD errors.
        """
        for idx, td_error in zip(batch_indices, batch_td_errors):
            self.priorities[idx] = (abs(td_error) + 1e-5) ** self.zeta

In [14]:
class QTrainer:
    def __init__(self, model, target_model, lr, gamma):
        self.model = model
        self.target_model = target_model
        self.gamma = gamma
        self.optimizer = keras.optimizers.Adam(learning_rate=lr)
        self.loss_fn = keras.losses.mean_squared_error
        
    @tf.function
    def train_step(self, states, actions, rewards, next_states, dones):
        next_Q_values = self.model(next_states)
        # Double DQN: model sceglie l'azione ma i Q-Value sono stimati da target_model
        max_next_Q_values_by_model = tf.argmax(next_Q_values, axis=1)
        mask_for_target = tf.one_hot(max_next_Q_values_by_model, 3)
        max_next_Q_values_by_target = tf.reduce_sum(self.target_model(next_states) * mask_for_target, axis=1)
        
        # Equazione di Bellman: Q value = reward + discount factor * expected future reward
        target_Q_values = rewards + (1 - dones) * self.gamma * max_next_Q_values_by_target
        with tf.GradientTape() as tape:
            all_Q_values = self.model(states)  
            Q_values = tf.reduce_sum(all_Q_values * actions, axis=1, keepdims=True)
            loss = tf.reduce_mean(self.loss_fn(target_Q_values, Q_values))
        # Backpropagation
        grads = tape.gradient(loss, self.model.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.model.trainable_variables))

In [15]:
# Espande di una dimensione. Quando si vuole (1,x) ma hai (1,)
def add_dimension(states, actions, rewards, next_states, dones):
    state = tf.expand_dims(states, axis=0)
    action = tf.expand_dims(actions, axis=0)
    reward = tf.expand_dims(rewards, axis=0)
    next_state = tf.expand_dims(next_states, axis=0)
    done = tf.expand_dims(dones, axis=0)
    return state, action, reward, next_state, done

def convert_to_tensorflow(states, actions, rewards, next_states, dones):
    states = tf.convert_to_tensor(states, dtype=tf.float32)
    actions = tf.convert_to_tensor(actions, dtype=tf.float32)
    rewards = tf.convert_to_tensor(rewards, dtype=tf.float32)
    next_states = tf.convert_to_tensor(next_states, dtype=tf.float32)
    dones = tf.convert_to_tensor(dones, dtype=tf.float32)
    return states, actions, rewards, next_states, dones

In [16]:
class Agent:
    def __init__(self, lr, gamma, max_memory, batch_size, nn_model):
        self.n_games = 0
        self.epsilon = 1  
        self.lr = lr
        self.gamma = gamma  
        self.memory = ReplayBuffer(max_size=max_memory)
        self.batch_size = batch_size
        self.model = nn_model
        self.target_model = keras.models.clone_model(self.model)
        self.trainer = QTrainer(self.model, self.target_model, lr=self.lr, gamma=self.gamma)

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((convert_to_tensorflow(state, action, reward, next_state, done)))

    def train_long_memory(self):
            states, actions, rewards, next_states, dones = self.memory.sample_experiences(self.batch_size)
            self.trainer.train_step(states, actions, rewards, next_states, dones)

    def train_short_memory(self, state, action, reward, next_state, done):
        state, action, reward, next_state, done = convert_to_tensorflow(state, action, reward, next_state, done)
        state, action, reward, next_state, done = add_dimension(state, action, reward, next_state, done)
        self.trainer.train_step(state, action, reward, next_state, done)

    def epsilon_greedy_policy(self, state):
        if np.random.rand() < self.epsilon:
            return np.random.randint(3)
        else:
            Q_values = self.model(state[np.newaxis])
            return np.argmax(Q_values[0])
        
    def get_action(self, state):
        final_move = [0, 0, 0]  
        move = self.epsilon_greedy_policy(state)
        final_move[move] = 1
        return final_move
    
    def train(self, N_GAME, visual=True):
        if visual:
            env = enviroment_visual.SnakeGameAI(speed=0)
        else:
            env = enviroment_no_visual.SnakeGameAI()
        plot_scores = []
        record = 0
        while self.n_games < N_GAME:
            state_old = env.get_state()
            final_move = self.get_action(state_old)
            state_new, reward, done, score = env.play_step(final_move)

            self.train_short_memory(state_old, final_move, reward, state_new, done)
            self.remember(state_old, final_move, reward, state_new, done)

            if done:
                env.reset()
                self.n_games += 1
                self.train_long_memory()
                print(f"\rGame: {self.n_games}, Epsilon: {self.epsilon:3f}, Score: {score}, Record: {record}.", end="")
                self.epsilon = max(((90 - self.n_games) / 90), 0)                                
                if score > record:
                    record = score
                plot_scores.append(score)
                
            # Accorgimento Fixed Q-Value Targets
            if self.n_games % 5 ==0:
                self.target_model.set_weights(self.model.get_weights())
        return plot_scores

Mostra andamento dello score per partita durante il training

In [17]:
def plot_trand(scores, save_path=None):
    zeros = [0 for i in range(49)]
    calcola_media = lambda i: sum(scores[i-10:i]) / 10
    media_precedenti = zeros + list(map(calcola_media, range(50, len(scores))))
    plt.plot(scores, label='Score')
    plt.plot(media_precedenti, label='Mean score delle ultime 50 partite')
    plt.text(len(media_precedenti)-1, media_precedenti[-1], f'{media_precedenti[-1]}', fontsize=12, color="darkorange", ha='center')

    plt.title("Andamento del training")
    plt.xlabel("Partite")
    plt.ylabel("Score")
    plt.legend()
    if save_path is not None:
        plt.savefig(save_path)
    plt.show()

Crea e allena un agente

In [18]:
model = Dueling_DQN(128)
agent = Agent(lr=0.001, gamma=0.9, max_memory=10_000, batch_size=1024, nn_model=model)
plot_scores = agent.train(N_GAME=150, visual=True)
plot_trand(plot_scores)

Game: 119, Epsilon: 0.000000, Score: 0, Record: 9.

KeyboardInterrupt: 